# Zeeman Dataverwerking
korte omschrijving, er zit commentaar bij dus het is duidelijk

nee waarschijnlijk niet helemaal, want het is mijn code dus ook mijn chaos

het voorbeeld is met dat plaatje zeeman tot zeeman_3.JPG uit de datasheet
hieer wordt gebruikgemaakt van 1 van de 2 kanalen van rgb (rood)

uit de gemeten data (fotos) bleek dat dat niet zo goed werkte met alleen rood, dus ik heb het grijsplaatje gepakt,


Ergens in een cell staat een functie, ik wilde alles in 1 functie zetten, maar daar schieten we niet heel veel meer mee op.
als er iets vreselijk vreemd of onduidelijk is, app me ff

## Importeer alle modules

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
%matplotlib inline

## Maak alle functies

In [ ]:
# functie zodat de plaatjes wat minder mini zijn
def display_img(img,cmap=None):
    """ 
    Displays the image in question:
    """
    fig = plt.figure(figsize=(14,24))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap)


def blur(img,blur):
    """
    Blurs the given image in grey and then returns it.
    """
    zm_origineel = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return cv2.medianBlur(zm_origineel,(blur)), zm_origineel

def r_img(path):
    """
    Reads the image from path and returns it:
    """
    return cv2.imread(path)

def sum_pixels(zm,length,axis="x"):
    """
    Sums up the desired pixel values on a vertical or horizontal basis:
    """
    y_tot = np.zeros(length)
    if axis == "x":
        for i in range(length):
            y_tot[i] = sum(zm[:,i])
        return y_tot
    
    elif axis == "y":
        for i in range(length):
            y_tot[i] = sum(zm[i,:])
        return y_tot

def find_mean_from_peaks(x,guess=600):
    """
    Vindt uit de piekdata het centrum door de dichtsbijzijnde pieken bij guess te pakken en deze te middelen:
    """
    peaks, _ = find_peaks(x, height=100,width=10)
    x_vals = peaks - guess
    x_max = x_vals[x_vals > 0][0]; x_min = x_vals[x_vals < 0][-1]
    x_peak = int(((x_max + x_min)/2) + guess)
    return x_peak, peaks

def find_nearest_two(array,value):
    """
    Finds the two closest neaghbours:
    """
    try:
        r_search = array[array > value][0] - value
    except:
        r_search = 0
    
    try:
        l_search = np.abs(array[array < value][-1] - value)
    except:
        l_search = 0

    return l_search, r_search

def main(path,i):
    """
    Does everything that is done below in one file swoop!
    """
    # niet aanzitten, dit is belangrijk
    # index  =[0  , 1 ,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]
    x_waardes = [0  ,639,639,639,639,639,639,639,639,639,639,639,639,639,639,639,639] #NVT
    y_waardes = [0  ,264,264,264,264,264,264,264,264,264,264,264,264,264,264,264,264] #NVT
    ksizes =  [1  , 13, 15, 15, 15, 13,  3,  3,  1,  3,  3,  3,  3,  5,  3,  5,  5] #done
    hights =  [1  , 80, 80, 80, 80, 80, 80, 80,100,110,110,110,100,100, 90, 90, 70] #done
    widths =  [1  ,  1,  1,  1,  1,  1,  1,  3,  1,  2,  1,  1,  1,  1,  2,  2,  1] #done
    dists  =  [1  ,  6,  6,  9,  9,  8,  7,  5,  5,  6,  8,  8,  7,  7,  8,  8,  6] #done
    #constanten
    i += 0
    res = 3.6e-6

    # import image
    img = r_img(path)
    zm_fout = img
    zm_gray = cv2.cvtColor(zm_fout,cv2.COLOR_BGR2GRAY)
    intensity_grayb = cv2.medianBlur(zm_gray,ksize = ksizes[i])
    zm_origineel, zm = blur(zm_fout,ksizes[i])
    zm_red = cv2.cvtColor(zm,cv2.COLOR_BGR2GRAY)

    # bereken de x en y richting van de pixels:
    y_lengte = np.shape(zm_red)[0]
    x_lengte = np.shape(zm_red)[1]

    # maak een lijst hiervan?
    ylin = np.linspace(0,y_lengte-1,y_lengte)
    xlin = np.linspace(0,x_lengte-1,x_lengte)

    # doe de som van de x-as om zo het coördinaat van de piek te vinden:
    x = sum_pixels(zm_red,y_lengte,axis="y"); x_tot=x
    # vind de twee pieken die het dichtst bij de 600 zitten en neem hiervaan het gemiddelde:
    evarage_guess = 250
    y_peak, peaks = find_mean_from_peaks(x,evarage_guess)

    #bereken de x_peaks values:
    # doe de som van de y-as om zo het coördinaat van de piek te vinden:
    x = sum_pixels(zm_red,x_lengte,axis="x"); y_tot=x
    # vind de twee pieken die het dichtst bij de 600 zitten en neem hiervaan het gemiddelde:
    evarage_guess = 600
    x_peak, x_peaks = find_mean_from_peaks(x,evarage_guess)

    #gebruik waardes van denise:
    x_peak = 639

    # bereken rm:
    # links:
    x_rel = x_peaks[x_peaks < x_peak] - x_peak
    x_relL = [[(((val*res)/(len(x_rel)-i)))**2,len(x_rel)-i] for i,val in enumerate(x_rel)]
    # rechts:
    x_rel = x_peaks[x_peaks > x_peak] - x_peak
    x_relR = [[(((val*res)/(i+1)))**2,i+1] for i,val in enumerate(x_rel)]
    # all:
    mx = []; [mx.append(x_relL[i][1]) for i in range(len(x_relL))]; [mx.append(x_relR[i][1]) for i in range(len(x_relR))]
    rm2 = []; [rm2.append(x_relL[i][0]) for i in range(len(x_relL))]; [rm2.append(x_relR[i][0]) for i in range(len(x_relR))]
    rm = x_peaks - x_peak
    rm2 =(np.abs(rm)*res)**2

    #x_peaks = x_peaks[mx > np.ones(len(mx))*2]
    #mx = mx[mx > np.ones(len(mx))*2]        

    # bepalen van lambda_0:
    f1 = 0.05
    f2 = 0.3
    d = 0.269
    f = ((1/f1)+(1/f2)-(d/(f1*f2)))**(-1)
    n = 1.456
    t = 0.003
    # lambda_0 is:
    lambda_0 = (np.polyfit(mx,rm2,deg=1)[0]*t*n)/(f**2)
    lambda_0 = lambda_0 * 10

    plt.scatter(mx,rm2,label="meetdata")
    p = np.poly1d(np.polyfit(mx,rm2,deg=1))
    plt.plot(mx,p(mx),label="fit:")
    plt.xlim([min(mx),max(mx)])
    plt.ylim([min(rm2),max(rm2)])
    plt.xlabel("orde [m]")
    plt.ylabel("$r_m^2$ [m$^2$]")
    plt.legend()
    plt.grid()
    plt.show()

    # vind alle pieken in de data
    y_strook = intensity_grayb[x_peak,:]; data = y_strook
    peaks, _ = find_peaks(y_strook,width=widths[i], height=hights[i],distance=dists[i])

    # select peaks:
    xpeaks = peaks
    s6 = xpeaks[9:21]
    s7 = xpeaks[7:19]
    s8 = xpeaks[9:27]
    s9 = xpeaks[6:21]
    s10 = xpeaks[4:19]
    s11 = xpeaks[6:24]
    s12 = xpeaks[8:29]
    s13 = xpeaks[6:24]
    s14 = xpeaks[5:29]
    s15 = xpeaks[6:30]
    s16 = xpeaks[4:31]
    afb  =['0','xpeaks','xpeaks','xpeaks','xpeaks','xpeaks','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16']

    # show a current plot:
    plt.figure(figsize=(10,5))
    plt.plot(data,label="dwarsdoorsnede pixelwaardes",color='blue')
    plt.scatter(xlin[eval(afb[i])],data[eval(afb[i])],color='red',label="peaks")
    plt.xlabel("x-as [pixel]")
    plt.ylabel("grijs-pixelwaarde [pixel]")
    plt.legend()
    plt.grid()
    plt.show()
    
    # vindt voor de peak in x_peaks twee nagelegen buren.
    dr = []
    for peak in x_peaks:
        dr.append(find_nearest_two(peaks,peak))
    dr1 = np.array(dr,dtype=float)*res

    # bereken dr en r en rdr
    dr = dr1.mean(axis=1)
    x_distance = (x_peaks - x_peak) * res
    r = x_distance
    rdr = np.abs(r) * np.abs(dr)

    x = mx
    y = [i[0] for i in dr1]
    plt.scatter(x,y)
    plt.xlim([min(x),max(x)])
    plt.ylim([min(y),max(y)])
    plt.show()

    return rdr, lambda_0



## Berekenen van bohr-magneton

In [ ]:
ksizes =  [1  , 13, 15, 15, 15, 13,  3,  3,  1,  3,  3,  3,  3,  5,  3,  5,  5] #done
hights =  [1  , 80, 80, 80, 80, 80, 80, 80,100,110,110,110,100,100, 90, 90, 70] #done
widths =  [1  ,  1,  1,  1,  1,  1,  1,  3,  1,  2,  1,  1,  1,  1,  2,  2,  1] #done
dists  =  [1  ,  6,  6,  9,  9,  8,  7,  5,  5,  6,  8,  8,  7,  7,  8,  8,  6] #done

with open("nu.txt","a+") as f:
    for i in range(len(ksizes)-1):
        i += 1
        f.write(str(ksizes[i])+","+str(hights[i])+","+str(widths[i])+","+str(dists[i])+"\n")
    f.close()

In [ ]:
afstanden = [54.5,53.5,52.5,51.5,50.5,49.5,48.5,47.5,46.5,45.5,44.5,43.5,42.5,41.5,40.5,40]
magneetveldsterktes = [200,210,300,350,380,410,450,540,660,750,740,920,1100,1500,1800,1900,1950]

B = np.array(magneetveldsterktes)
h = 6.626e-34
c = 3e9

# DO ALL IN ONE!

In [ ]:
# get the names of all pictures:
import glob
all_pics = glob.glob("zeeman\zeeman\WIN*.jpg")

In [ ]:
# do this for all pictures:
lijst = []
for i,pic in enumerate(all_pics):
    lijst.append(main(pic,i))

# Print voor alle afbeeldingen de lambda_0's:

In [ ]:
img = r_img(all_pics[6])
zm_fout = img
zm_gray = cv2.cvtColor(zm_fout,cv2.COLOR_BGR2GRAY)
plt.figure(figsize=(10,5))
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.xlabel("x-as [pixels]")
plt.ylabel("y-as [pixels]")
plt.show()

In [ ]:
len(all_pics)-1

In [ ]:
_=[print("lambda_0 "+str(i)+": ",val[1]*1e9) for i,val in enumerate(lijst)]
print(np.array([val[1] for i,val in enumerate(lijst)]).mean())
print(np.array([val[1] for i,val in enumerate(lijst)]).std())

In [ ]:
ljs = []
for i,val in enumerate(lijst):
    rdr = lijst[i][0]
    lambda_0 = lijst[i][1]
    
    mu_b = (val[0].mean() * h * c)/((lambda_0**2)*(B[i]/1000))
    ljs.append(mu_b*1e24)
    print(mu_b*1e24)
print("\n\n\n")
print(np.array(ljs).mean())
print(np.array(ljs).std())

In [ ]:
np.array(ljs).mean()
np.array(ljs).std()

In [ ]:
[value[i][3] for i,value in enumerate(lijst)]

In [ ]:
plt.scatter(magneetveldsterktes,np.array(lijst).mean(axis=1))